<a href="https://colab.research.google.com/github/Mikhail-068/NetOptic/blob/master/Tretyakov_Aleksandr/1_Creating_a_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Подключение модуля для загрузки данных из облака
import gdown
# Модуль для работы с файлами операционной системы
import os
from bs4 import BeautifulSoup
# Модуль для вывода данных в читабельном виде
from pprint import pprint
import pandas as pd
import numpy as np

# Скачивание базы

Чтобы постоянно не долбиться на сервер, скачаем базу один раз, сохраним её на диск и потом будем обращаться к файлу.

In [ ]:
DOWNLOAD_LINKS = {'с сервера': 'https://www.netoptik.ru/yandexmarket.yml',
                 'с google.drive': 'https://drive.google.com/uc?id=12Feyswg1yBGfUvJKLvWtV-OIP-HbUqME'
                 }
DOWNLOADS_METHOD = 'с google.drive'

base_file_name = 'yandexmarket.yml'
gdown.download(DOWNLOAD_LINKS[DOWNLOADS_METHOD], output=base_file_name)
base_path = os.path.join('/content', base_file_name)



Downloading...
From: https://drive.google.com/uc?id=12Feyswg1yBGfUvJKLvWtV-OIP-HbUqME
To: /content/yandexmarket.yml
100%|██████████| 4.55M/4.55M [00:00<00:00, 96.3MB/s]


# Парсинг

In [ ]:
!pip install beautifulsoup4==4.12.2 lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2


In [ ]:
# читаем файл
with open(base_path, 'r') as file:
    xml_file = file.read()

In [ ]:
# Получаем объект bs
soup = BeautifulSoup(xml_file, 'lxml')
offers = soup.findAll("offer")

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


## Эксперименты с парсером

In [ ]:
# Номер ордера, который мы ходим посмотреть
OFFER_NUM = 12
one_offer = offers[OFFER_NUM]
print(one_offer)

<offer id="11795">
<categoryid>177</categoryid>
<name>Оправа Gucci, GG 1006, 4GX</name>
<url>https://www.netoptik.ru/frames/men-frames/oprava-gucci-gg1006-4gx</url>
<price>7500</price>
<currencyid>RUB</currencyid>
<vendor>GUCCI</vendor>
<model>Оправа Gucci, GG 1006, 4GX</model>
<picture>https://www.netoptik.ru/image/cache/catalog/Gucci%20(ЮД)/gucci-gg1006-4gx-s2-1%20(2)-600x600.jpg</picture>
<picture>https://www.netoptik.ru/image/cache/catalog/Gucci%20(ЮД)/gucci-gg1006-4gx-s2-2%20(1)-600x600.jpg</picture>
<picture>https://www.netoptik.ru/image/cache/catalog/Gucci%20(ЮД)/gucci-gg1006-4gx-s2-3%20(3)-600x600.jpg</picture>
<param name="Категория"/>Мужские оправы для очков
<param name="Ширина оправы (мм)"/>135
<param name="Ширина линзы (мм)"/>54
<param name="Высота линзы (мм)"/>36
<param name="Мост (мм)"/>16
<param name="Длина заушника (мм)"/>140
<param name="Материал "/>Комбинированный
<param name="Стиль"/>Мужские
<param name="Строение оправы"/>Ободковая
<param name="Цвет оправы"/>Черепахо

In [ ]:
# Посмотрим названия всех доступных параметров
for param in one_offer.select('param'):
    print(param.attrs['name'])

Категория
Ширина оправы (мм)
Ширина линзы (мм)
Высота линзы (мм)
Мост (мм)
Длина заушника (мм)
Материал 
Стиль
Строение оправы
Цвет оправы
Форма очков


## Парсим базу

In [ ]:
# Зададим список данных (тэгов), которые нам нужны
TAGS_LIST = ['name', 'picture']
# Зададим список параметров, которые нам нужны
PARAMS_LIST = ['Категория', 'Материал']

In [ ]:
data_list = []

for offer in offers:
    new_line = {}

    # Добавляем значения нужных нам тэгов
    for tag in TAGS_LIST:
        new_line[tag] = offer.find(tag).getText()

        if tag == 'picture':
            result = []
            for pic in offer.find_all(tag):
                result.append(pic.getText())
            new_line[tag] = '\n'.join(result)


    # Добавляем значения нужных нам параметров
    for needed_param in PARAMS_LIST:
        for param in offer.find_all('param'):
            if needed_param in param.attrs['name'].strip():
                new_line[needed_param] = param.next_element.replace('\n', '').strip()

    # добавляем данные в список data_list
    data_list.append(new_line)

In [ ]:
# Добавляем данные в датафрейм
base_df = pd.DataFrame(data_list)

In [ ]:
display(base_df)

,name,picture,Категория,Материал
0,"Оправа Balenciaga, BAL 0108, 8O0",https://www.netoptik.ru/image/cache/data/med_o...,Оправы для очков,комбинированный
1,"Оправа Boss Orange, BO 0329, YZ4",https://www.netoptik.ru/image/cache/catalog/Bo...,Мужские оправы для очков,Металл
2,"Оправа Boss Hugo Boss, BOSS 1312, 003",https://www.netoptik.ru/image/cache/catalog/Bo...,Оправы для очков,Комбинированный
3,"Оправа Boss Hugo Boss, BOSS 1246, CNI",https://www.netoptik.ru/image/cache/catalog/Bo...,Оправы для очков,Металл
4,"Оправа Carrera, CA 6660, VBP",https://www.netoptik.ru/image/cache/data/carre...,Женские оправы для очков,металл
...,...,...,...,...
2448,"Оправа FILOS, FF1300, 04",https://www.netoptik.ru/image/cache/catalog/FI...,NaN,Металл
2449,"Оправа FILOS, FF1300, 05",https://www.netoptik.ru/image/cache/catalog/FI...,NaN,Металл
2450,"Оправа FILOS, FF1303, 03",https://www.netoptik.ru/image/cache/catalog/FI...,NaN,Металл
2451,"Оправа FILOS, FF1303, 01",https://www.netoptik.ru/image/cache/catalog/FI...,NaN,Металл


In [ ]:
def show_unique_values(column, lower=False):
    """
    Функция показывает все возможные значения из заданной колонки.
    :param column: Название колонки.
    :param lower: Приводить ли значения к нижнему регистру.
    :return: none
    """
#    unique_values = base_df['Категория'].fillna('нет данных').unique()
    if lower:
        print(base_df[column].fillna('нет данных').str.lower().value_counts().sort_index())
    else:
        print(base_df[column].fillna('нет данных').value_counts().sort_index())


### Посмотрим на значения в колонке "Материал"

In [ ]:
show_unique_values(column='Материал', lower=False)
print('='*30)
show_unique_values(column='Материал', lower=True)

Silflex                    8
Комбинированные            5
Комбинированный          417
Металл                   317
Пластик                  965
Пластик, ударопрочный      1
Поликарбонат              23
Силикон                   71
Стекло                     1
Титан                     65
Трайвекс                   5
комбинированный          207
металл                   112
нет данных                19
пластик                  210
силикон                   24
титан                      3
Name: Материал, dtype: int64
silflex                     8
комбинированные             5
комбинированный           624
металл                    429
нет данных                 19
пластик                  1175
пластик, ударопрочный       1
поликарбонат               23
силикон                    95
стекло                      1
титан                      68
трайвекс                    5
Name: Материал, dtype: int64


# Причешем данные

## Обрабатываем колонку "name"

In [ ]:
def processing_name():
    """
    Функция удаляет из колонки 'name' ненужные нам позиции.
    :return: none
    """
    global base_df

    # Список удаляемых значений
    delete_list = ['линза', 'Линза', 'Pensne', 'Горнолыжная']

    # создаем маску на основе поиска подстроки
    mask = base_df['name'].str.contains('|'.join(delete_list))

    # удаляем строки, где значение в столбце 'name' содержит хотя бы
    # одно значение из списка delete_list
    base_df = base_df[~mask]

In [ ]:
processing_name()

## Обрабатываем колонку "Категория"

### Посмотрим на значения в колонке 'Категория'



In [ ]:
show_unique_values(column='Категория', lower=False)

Аксессуары                    1
Водительские очки             4
Готовые очки                  2
Детские                     153
Детские очки                  2
Детский                       1
Женские оправы для очков    199
Женские очки                 96
Женскиеоправы для очков       1
Женский                       1
Компьютерные очки             6
Линзы для очков               1
Мужские оправы для очков    100
Мужские очки                 42
Мужскиеоправы для очков       1
Оправы                        1
Оправы для очков            368
Оправы оправы для очков       1
Очки для плавания             2
Очки на очки                  5
Очки спецназначения           3
Очки тренажеры                2
Пенсне                        3
Половинки для чтения          1
Солнцезащитные очки         122
Спецназначения                2
Спортивные                    1
Спортивные очки               6
Унисекс                       8
женские очки                  1
нет данных                  731
Name: Ка

In [ ]:
def processing_category():
    """
    Функция приводит значения из колонки "Категория" к однообразию, удаляет ненужные
    строки, делает необходимые переименования.
    :return: none
    """
    global base_df

    # nan заменим на 'нет данных'
    base_df['Категория'] = base_df['Категория'].fillna('нет данных')

    # Удалим ненужные данные
    delete_list = ['Аксессуары', 'Водительские очки', 'Бифокальные линзы',
                    'Компьютерные линзы','Линзы для детей','Линзы для очков',
                    'Линзы с поддержкой аккомадации','Однофокальные линзы',
                    'Офисные линзы','Прогрессивные линзы','Фотохромные линзы',
                    'Пенсне', 'Очки тренажеры', 'SALE', 'Лентикуляры для высоких диоптрий',
                'Очки для плавания']

    # удаляем строки, у которых значение в столбце "Категория" равно
    # значению из списка delete_list
    base_df = base_df[~base_df['Категория'].isin(delete_list)]

    # Словарь переименований
    replace_dict = {
        #'аксессуары': ['Аксессуары'],
        'бифокальные линзы': ['Бифокальные линзы'],
        'водительские очки': ['Водительские очки'],
        'готовые очки': ['Готовые очки'],
        'детские очки': ['Детские очки', 'Детские', 'Детский'],
        'женские оправы для очков': ['Женские оправы для очков', 'Женскиеоправы для очков'],
        'женские очки': ['Женские очки', 'Женский', 'женские очки'],
        'компьютерные линзы': ['Компьютерные линзы'],
        'компьютерные очки': ['Компьютерные очки'],
        'лентикуляры для высоких диоптрий': ['Лентикуляры для высоких диоптрий'],
        'линзы для детей': ['Линзы для детей'],
        'линзы для очков': ['Линзы для очков'],
        'линзы с поддержкой аккомадации': ['Линзы с поддержкой аккомадации'],
        'мужские очки': ['Мужские очки'],
        'мужские оправы для очков': ['Мужские оправы для очков', 'Мужскиеоправы для очков'],
        'однофокальные линзы': ['Однофокальные линзы'],
        'оправы для очков': ['Оправы для очков', 'Оправы', 'Оправы оправы для очков'],
        'офисные линзы': ['Офисные линзы'],
        'очки для плавания': ['Очки для плавания'],
        'очки на очки': ['Очки на очки'],
        'очки спецназначения': ['Очки спецназначения', 'Спецназначения'],
        'очки тренажеры': ['Очки тренажеры'],
        'пенсне': ['Пенсне'],
        'половинки для чтения': ['Половинки для чтения'],
        'прогрессивные линзы': ['Прогрессивные линзы'],
        'cолнцезащитные очки': ['Солнцезащитные очки'],
        'спортивные очки': ['Спортивные очки', 'Спортивные'],
        'унисекс': ['Унисекс'],
        'фотохромные линзы': ['Фотохромные линзы']
        }

    # заменяем значения в столбце "Категория" по словарю replace_dict
    for key, values in replace_dict.items():
        base_df.loc[base_df['Категория'].isin(values), 'Категория'] = key

    # Обработка позиций, у которых не указана категория


    # Вывод результата
    show_unique_values(column='Категория')

In [ ]:
processing_category()

cолнцезащитные очки         122
готовые очки                  2
детские очки                156
женские оправы для очков    200
женские очки                 98
компьютерные очки             6
мужские оправы для очков    101
мужские очки                 42
нет данных                  731
оправы для очков            370
очки на очки                  5
очки спецназначения           5
половинки для чтения          1
спортивные очки               7
унисекс                       8
Name: Категория, dtype: int64


## Обрабатываем колонку "Материал"

In [ ]:
def processing_materials():
    """
    Функция приводит значения из колонки 'Материал' к однообразию, удаляет ненужные
    строки, делает необходимые переименования.
    :return: none
    """
    global base_df

    # nan заменим на 'нет данных'
    base_df['Материал'] = base_df['Материал'].fillna('нет данных')

    # Удалим ненужные данные
    delete_list = ['Silflex', 'Пластик, ударопрочный', 'Поликарбонат',
                   'Силикон', 'силикон', 'Стекло', 'титан', 'Титан',
                   'Трайвекс']

    # удаляем строки, у которых значение в столбце 'Материал' равно
    # значению из списка delete_list
    base_df = base_df[~base_df['Материал'].isin(delete_list)]

    # Словарь переименований
    replace_dict = {
        'комбинированный': ['Комбинированные', 'Комбинированный'],
        'металл': ['Металл'],
        'пластик': ['Пластик']
        #'silflex': ['Silflex'],
        #'поликарбонат': ['Поликарбонат'],
        #'силикон': ['Силикон'],
        #'стекло': ['Стекло'],
        #'титан': ['Титан'],
        #'трайвекс': ['Трайвекс']
        }

    # заменяем значения в столбце 'Материал' по словарю replace_dict
    for key, values in replace_dict.items():
        base_df.loc[base_df['Материал'].isin(values), 'Материал'] = key

    # Обработка позиций, у которых не указан материал
    # Создаем маску для поиска значений
    mask = base_df['name'].isin(['Оправа Giorgio Armani, GA 890, XZW',
                                'Оправа Giorgio Armani, GA 941, 086',
                                'Оправа Gucci, GG 3825 KCL',
                                'Оправа Juicy Couture, SPLASHBACK V05'])

    # Заменяем значения в столбце 'Материал' на 'пластик' для
    # соответствующих значений в столбце 'name'
    base_df.loc[mask, 'Материал'] = 'пластик'


    # Вывод результата
    show_unique_values(column='Материал')

In [ ]:
processing_materials()

комбинированный    626
металл             419
пластик            637
Name: Материал, dtype: int64


<ipython-input-19-64628e69bd57>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df.loc[base_df['Материал'].isin(values), 'Материал'] = key
<ipython-input-19-64628e69bd57>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df.loc[mask, 'Материал'] = 'пластик'


# Сохранение датасета в файл

In [ ]:
# Сохраним данные в файл формата csv
base_df.to_csv('materials_data.csv', index=False)